In [1]:
import pandas as pd

final_df = pd.read_csv('../dataset/final_dataset.csv')

In [2]:
final_df

,id,velocity,Period,Cycle_Number,experiment_number,position,displacement,smoothed_displacement
0,0,160.0,300.65650,1772.25,4.5,0.0,0.000203,0.000177
1,1,160.0,601.11325,3009.75,4.5,0.0,0.000196,0.000172
2,2,160.0,901.65225,4246.75,4.5,0.0,0.000193,0.000171
3,3,160.0,1202.14075,5483.50,4.5,0.0,0.000195,0.000173
4,4,160.0,1502.75775,6721.00,4.5,0.0,0.000193,0.000173
...,...,...,...,...,...,...,...,...
1191,1191,380.0,19230.41400,188809.50,6.5,1.0,0.000232,0.000205
1192,1192,380.0,19530.91400,191746.00,6.5,1.0,0.000242,0.000208
1193,1193,380.0,19831.52350,194684.50,6.5,1.0,0.000237,0.000207
1194,1194,380.0,20132.34350,197626.00,6.5,1.0,0.000238,0.000205


# Data prepare

In [3]:
final_160 = final_df[final_df['velocity'] == 160]
split_index = int(0.8 * len(final_160))
print(len(final_160))
print(split_index)

322
257


In [17]:
final_160 = final_160.sort_values('Cycle_Number')

split_index = int(0.8 * len(final_160))

train_data_160 = final_160.iloc[:split_index]
test_data_160 = final_160.iloc[split_index:]

X_train_160 = train_data_160[['Cycle_Number', 'velocity']]
y_train_160 = train_data_160['displacement']

X_test_160 = test_data_160[['Cycle_Number', 'velocity']]
y_test_160 = test_data_160['displacement']


In [18]:
X_test_160

,Cycle_Number,velocity
128,160125.75,160.0
290,161363.25,160.0
129,161363.25,160.0
291,162600.75,160.0
130,162600.75,160.0
...,...,...
158,197242.75,160.0
320,198479.50,160.0
159,198479.50,160.0
160,199717.25,160.0


# GradientBoostingRegressor

In [19]:
from sklearn.ensemble import GradientBoostingRegressor

forecasting_model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)

forecasting_model.fit(X_train_160, y_train_160)


GradientBoostingRegressor(max_depth=5, random_state=42)

In [21]:
# Predict on the future cycles
y_pred_test_160 = forecasting_model.predict(X_test_160)

# Evaluate the model performance
from sklearn.metrics import mean_absolute_error, r2_score
mae_test_160 = mean_absolute_error(y_test_160, y_pred_test_160)
r2_test_160 = r2_score(y_test_160, y_pred_test_160)

print(f'Mean Absolute Error on Future Cycles: {mae_test_160:.4f}')
print(f'R² Score on Future Cycles: {r2_test_160:.4f}')


Mean Absolute Error on Future Cycles: 0.0000
R² Score on Future Cycles: -0.6123


In [23]:
# Create a DataFrame for future cycles
future_cycles = pd.DataFrame({
    'Cycle_Number': range(200569, 210000),  # Example: forecasting up to cycle 210,000
    'velocity': [final_df['velocity'].iloc[-1]] * (210000 - 200569)  # Use the last known velocity
})

# Predict displacement for these future cycles
future_displacement_predictions = forecasting_model.predict(future_cycles)

# Add the predictions to the DataFrame
future_cycles['predicted_displacement'] = future_displacement_predictions

# Display the future predictions
future_cycles


,Cycle_Number,velocity,predicted_displacement
0,200569,380.0,0.000186
1,200570,380.0,0.000186
2,200571,380.0,0.000186
3,200572,380.0,0.000186
4,200573,380.0,0.000186
...,...,...,...
9426,209995,380.0,0.000186
9427,209996,380.0,0.000186
9428,209997,380.0,0.000186
9429,209998,380.0,0.000186
